In [ ]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)

#provide the assisatne id bellow
assistant = client.beta.assistants.retrieve("")


In [ ]:
def ask_assistant(question):
    thread = client.beta.threads.create()
    
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    
    while run.status != 'completed':
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages.data[0].content[0].text.value

In [ ]:
answer = ask_assistant("when is the final exam ")
print(answer)

In [ ]:
import win32com.client

# Connect to Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

# Access the Inbox folder
inbox = outlook.GetDefaultFolder(6)  # 6 is the index for the Inbox folder
messages = inbox.Items

# Sort messages by received time (descending)
messages.Sort("[ReceivedTime]", True)

# Filter emails based on sender address
def filter_emails_by_sender(sender_email):
    filtered_emails = []
    for message in messages:
        try:
            if message.SenderEmailAddress.lower() == sender_email.lower():
                filtered_emails.append(message)
        except Exception as e:
            # Handle messages that don't have a sender or are corrupted
            print(f"Error processing email: {e}")
    return filtered_emails

# Extract sender name and body text
def extract_email_details(message):
    try:
        sender_name = message.SenderName
        body = message.Body
        return sender_name, body
    except Exception as e:
        print(f"Error extracting email details: {e}")
        return None, None

# Create a reply draft
def create_reply_draft(message, reply_body):
    try:
        reply = message.Reply()
        reply.Body = f"{reply_body}\n\n--- Original Message ---\n{message.Body}"
        reply.Save()  
    except Exception as e:
        print(f"Error creating reply draft: {e}")


In [ ]:
sender_email_to_filter = "weixuebin@gmail.com"  # Replace with the email address to filter
filtered_emails = filter_emails_by_sender(sender_email_to_filter)

if filtered_emails:
    for email in filtered_emails[:1]:  # Get only the latest matching email
        sender_name, body = extract_email_details(email)
        if sender_name and body:
            print(f"Sender Name: {sender_name}")
            print(f"Email Body:\n{body}")
            
            # Create a reply draft
            reply_body = ask_assistant(f"sender: {sender_name}, body:{body}")
            create_reply_draft(email, reply_body)
else:
    print("No emails found from the specified sender.")